This example shows a basic example of "Feff fitting" with larch for Cu metal at room temperature.

We start with some import statements, then do a "normal" first-shell fit
   1. read in data for Cu metal, Cu K edge
   2. do background subtraction and plot $\chi(k)$
   3. set the Fourier transform ranges
   4. define a Group of Parameters for the fit
   5. read in 1 Feff Path for Cu-Cu scattering
   6. run the fit and view the results
   
Then we will expand that to include 2 more paths, and show one way to constrain the distances parameters

In [1]:
# step 0: imports
import numpy as np
from pathlib import Path
from larch.xafs import (pre_edge, autobk, xftf, xftr,
                        ff2chi, feffpath, feffit_transform, 
                        feffit_dataset, feffit, feffit_report)
from larch.fitting import param, guess, param_group
from larch.io import read_ascii
from larch.plot.bokeh_xafsplots import (plot_mu,plot_chik, plot_chir, plot_chifit, 
                                         multi_plot, plotlabels, set_label_weight)

In [2]:
# step 1: read in the data and make mu(E) for room Temperature Cu

cu_data  = read_ascii(Path('..', 'xafsdata', 'cu_metal_rt.xdi'), labels='energy i0 itrans ifluor')
cu_data.mu = -np.log(cu_data.itrans/cu_data.i0)
pre_edge(cu_data)
fig = plot_mu(cu_data, show_norm=True, show_e0=True)


Loading BokehJS ...

In [3]:
# step 2: do and check the background subtraction
autobk(cu_data, rbkg=1.0, kweight=2)
fig = plot_chik(cu_data, kweight=2)

Loading BokehJS ...

In [4]:
# step 3: Fourier transform ranges ranges
xftf(cu_data, kmin=3, kmax=17, kweight=2, dk=4, window='kaiser')
xftr(cu_data, rmin=1.2, rmax=3, window='hanning', dr=0)

plot_chir(cu_data, show_window=True, rmax=7)


# we can then definte Fourier tranform and fit ranges (first shell only: out to 3.0 Ang)
trans = feffit_transform(kmin=3, kmax=17, kweight=2, dk=4, window='kaiser', rmin=1.2, rmax=3.0)

Loading BokehJS ...

In [5]:
# step 4: setup fitting Parameters: we will vary 
#   amplitude, e0, sigma2, and delta_R, and a third cumulant C3

pars = param_group(amp=param(1, vary=True),
                   de0=param(3, vary=True),
                   c3=param(.0002, vary=True),
                   sig2=param(.005, vary=True),
                   delr=param(0.01, vary=True) )


In [6]:
# step 5: define a Feff Path for Cu-Cu scattering

# using a Path file from a previous Feff calculation for Zn-Se (shown elsewhere)
# here we give math expressions using the Parameters named in the Parameter group 
# to set the values for the Path Parameters in the EXAFS equation
path1 = feffpath(Path('..', 'feffit','feff0001.dat'), label='Cu R=2.56Ang',
            s02='amp', e0='de0', sigma2='sig2', deltar='delr', third='c3')

tmpsum = ff2chi([path1], params=pars)
fig = plot_chik(tmpsum, kweight=2, label='initial Feff Model', show=False)
fig.add_plot(cu_data.k, cu_data.chi*cu_data.k**2, label='data')
fig.show()

Loading BokehJS ...

In [7]:
# step 6: run and view the fit

# now that we have data, a set of paths, and Fourier transform values, we can
# set up a "Feffit Dataset" and then fit that with the group of Parameters

# define dataset to include data, pathlist, transform
dset = feffit_dataset(data=cu_data, pathlist=[path1], transform=trans)

# do the fit...
result = feffit(pars, dset)

# print out the fit report
print(feffit_report(result))

=================== FEFFIT RESULTS ====================
[[Statistics]]
  n_function_calls     = 31
  n_variables          = 5
  n_data_points        = 116
  n_independent        = 17.0428183
  chi_square           = 73.8303606
  reduced chi_square   = 6.13065472
  r-factor             = 0.00177931
  Akaike info crit     = 34.9854727
  Bayesian info crit   = 39.1641172
 
[[Parameters]]
  amp                  =  0.9425256 +/- 0.0312341  (init= 1.0000000)
  c3                   =  2.0191e-4 +/- 7.6522e-5  (init= 2.0000e-4)
  de0                  =  6.0481827 +/- 0.7342105  (init= 3.0000000)
  delr                 =  0.0118328 +/- 0.0069547  (init= 0.0100000)
  sig2                 =  0.0087457 +/- 2.4353e-4  (init= 0.0050000)
 
[[Correlations]] (unreported correlations are <  0.100)
  c3, delr             =  0.957
  de0, delr            =  0.951
  amp, sig2            =  0.929
  c3, de0              =  0.846
  amp, delr            =  0.184
  delr, sig2           =  0.171
  amp, c3        

In [8]:
# finally, plot the results
fig6, fig7 = plot_chifit(result.datasets[0])

Loading BokehJS ...

Loading BokehJS ...

From the fit report,we see that the third cumulant, $C_3$ is ever-so-slightly above 0 for room temperature Cu, and that this is highly correlated with $R$.   In another notebook, we'll explore those correlations and uncertainties
in more detail.

Now we can add two more Paths for Cu scattering, and extend the R-range of the fit range to 3.5 Ang

In [9]:
# we will just change the Rmax to 3.5:
trans.rmax  = 3.5

# and we will update the parameters to 
# include a separate sigma2 for each path, 
# but we'll change 'delr' to 'alpha' and use this as fractional change in distance for each path.
pars = param_group(amp=param(1,vary=True),
             de0  = param(3,    vary=True),
             c3   = param(.002, vary=True),
             sig2 = param(.002, vary=True),
             sig2_p2 = param(.002, vary=True),
             sig2_p3 = param(.002, vary=True),
             alpha= param(0.001, vary=True) )


# and we will define 3 paths to use use these:
path1.deltar = 'alpha*reff'

path2 = feffpath('../feffit/feff0002.dat',  s02='amp', e0='de0',
                 sigma2='sig2_p2',  deltar='alpha*reff')

path3 = feffpath('../feffit/feff0003.dat', s02='amp', e0='de0',
                  sigma2='sig2_p3', deltar='alpha*reff')

# note that all 3 expressions for 'deltar' are the same 'alpha*reff'.  
# The value of 'reff' will be nominal Path distance

In [10]:
# now we can run that fit and view the results

dset  = feffit_dataset(data=cu_data, pathlist=[path1, path2, path3], transform=trans)

result = feffit(pars, dset)

print(feffit_report(result))

=================== FEFFIT RESULTS ====================
[[Statistics]]
  n_function_calls     = 107
  n_variables          = 7
  n_data_points        = 150
  n_independent        = 21.4991567
  chi_square           = 104.993309
  reduced chi_square   = 7.24133903
  r-factor             = 0.00253915
  Akaike info crit     = 48.0951453
  Bayesian info crit   = 55.5712412
 
[[Parameters]]
  alpha                =  0.0027937 +/- 0.0027987  (init= 1.0000e-3)
  amp                  =  0.9355016 +/- 0.0340352  (init= 1.0000000)
  c3                   =  1.4437e-4 +/- 7.7277e-5  (init= 0.0020000)
  de0                  =  5.6958140 +/- 0.7689779  (init= 3.0000000)
  sig2                 =  0.0086826 +/- 2.6711e-4  (init= 0.0020000)
  sig2_p2              =  0.0131224 +/- 0.0010719  (init= 0.0020000)
  sig2_p3              =  0.0062715 +/- 0.0028923  (init= 0.0020000)
 
[[Correlations]] (unreported correlations are <  0.100)
  alpha, c3            =  0.952
  alpha, de0           =  0.950
  amp,

In [11]:
# and plot the results
fig = plot_chifit(result.datasets[0], show_real=True)

Loading BokehJS ...

Loading BokehJS ...

In [12]:
from larch.plot.bokeh_xafsplots import plot_paths_k, plot_paths_r

plot_paths_k(result.datasets[0], offset=-0.25)

Loading BokehJS ...